In [1]:
import numpy as np
import torch
import torch.nn as nn
import os, time
from skimage.measure import compare_psnr, compare_ssim
from skimage.io import imread, imsave, imshow
import cv2

In [23]:
def save_result(result, path):
    path = path if path.find('.') != -1 else path+'.png'
    ext = os.path.splitext(path)[-1]
    if ext in ('.txt', '.dlm'):
        np.savetxt(path, result, fmt='%2.4f')
    else:
        imsave(path, np.clip(result, 0, 1))

def save_residual(r, path):
    path = path if path.find('.') != -1 else path+'.png'
    ext = os.path.splitext(path)[-1]
    
    r = 2*(r+0.4)-0.3
    imsave(path, np.clip(r, 0, 1))

def save_structure(s, path):
    path = path if path.find('.') != -1 else path+'.png'
    ext = os.path.splitext(path)[-1]
    
    #s = 4*(s+0.3)-0.7
    s = s
    imsave(path, np.clip(s, 0, 1))

def show(x, title=None, cbar=False, figsize=None):
    import matplotlib.pyplot as plt
    plt.figure(figsize=figsize)
    plt.imshow(x, interpolation='nearest', cmap='gray')
    if title:
        plt.title(title)
    if cbar:
        plt.colorbar()
    plt.show()

In [38]:
set_dir = 'data/Test'
set_names = ['Set12']
sigma =25
model_dir = 'models'
model_name = 'dncnn25(17).pth'
result_dir = 'results'

model = torch.load(os.path.join(model_dir, model_name))

model.eval()

print(model)
if torch.cuda.is_available():
    model = model.cuda()

for set_cur in set_names:

    if not os.path.exists(os.path.join(result_dir, set_cur)):
        os.mkdir(os.path.join(result_dir, set_cur))
    psnrs = []
    ssims = []

    for im in os.listdir(os.path.join(set_dir, set_cur)):
        if im.endswith(".jpg") or im.endswith(".bmp") or im.endswith(".png"):
            
            x = np.array(imread(os.path.join(set_dir, set_cur, im)), dtype=np.float32)/255.0
            init_shape = x.shape
                
            np.random.seed(seed=0)  # for reproducibility
            y = x + np.random.normal(0, sigma/255.0, x.shape)  # Add Gaussian noise without clipping
            y = y.astype(np.float32)
            y_ = torch.from_numpy(y).view(1, -1, y.shape[0], y.shape[1])

            torch.cuda.synchronize()
            start_time = time.time()
            y_ = y_.cuda()
            r = model(y_)
            x_ = y_ - r  # inference
            
            '''x_zero = torch.zeros(x_.size()).type(x_.type())
            x_temp = torch.cat((x_,x_zero),1) #[-1,patch_size, patch_size, 2]
            print(x_temp.size())
            x_fft = torch.fft(x_temp.permute(0,2,3,1), 2) #fast fourier transform
            x_fft = torch.ifft(x_fft, 2)[:,:,:,1]'''
            
            x_ = x_.view(y.shape[0], y.shape[1])
            x_ = x_.cpu()
            x_ = x_.detach().numpy().astype(np.float32)
            
            r = r.view(y.shape[0], y.shape[1])
            r = r.cpu()
            r = r.detach().numpy().astype(np.float32)
            
            x_fft1 = x_fft1.view(y.shape[0], y.shape[1]).cpu().detach().numpy.astype(np.float32)
            
            x_fft = np.fft.fft2(r)
            fshift = np.fft.fftshift(x_fft)
            magnitude_spectrum = 20*np.log(np.abs(fshift))
            
            print(np.array_equal(x_fft1, x_fft))
            
            torch.cuda.synchronize()
            elapsed_time = time.time() - start_time
            print('%10s : %10s : %2.4f second' % (set_cur, im, elapsed_time))

            psnr_x_ = compare_psnr(x, x_)
            ssim_x_ = compare_ssim(x, x_)
            
            name, ext = os.path.splitext(im)
            show(np.hstack((y, x_)))  # show the image
            save_result(x_, path=os.path.join(result_dir, set_cur, name+'_d1'+ext))  # save the denoised image
            save_residual(r, path=os.path.join(result_dir, set_cur, name+'_residual'+ext))  # save the denoised image
            save_structure(magnitude_spectrum, path=os.path.join(result_dir, set_cur, name+'_fft'+ext))  # save the denoised image
            
            psnrs.append(psnr_x_)
            ssims.append(ssim_x_)
    psnr_avg = np.mean(psnrs)
    ssim_avg = np.mean(ssims)
    psnrs.append(psnr_avg)
    ssims.append(ssim_avg)

    print('Datset: {0:10s} \n  PSNR = {1:2.2f}dB, SSIM = {2:1.4f}'.format(set_cur, psnr_avg, ssim_avg))

DNet(
  (dncnn): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (4): ReLU(inplace)
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (6): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (7): ReLU(inplace)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (12): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddin

RuntimeError: Expected an input tensor with a last dimension of size 2 representing real + imaginary components, but got input torch.cuda.FloatTensor[1, 1, 256, 256]